# Preparing

In [2]:
import numpy as np
from tqdm import tqdm

import tensorflow as tf
import keras
from keras.preprocessing import timeseries_dataset_from_array
from keras.callbacks import CSVLogger
from sklearn.metrics import classification_report

import data
from modules.utilites import DataClass

seq_len = 100

In [3]:
# Load data
row_data = (
    # data.load_dataset(horizon=4)
    data.load_saved_datas())
data.inspect_datas(row_data)

    Datas:
train     : x= (203799, 40)    | y= (203799,)      
val       : x= (50949, 40)     | y= (50949,)       
test      : x= (139586, 40)    | y= (139586,)      


In [4]:
# # Save data
# data.save_data(name= 'train',x= x_train,y=y_train)
# data.save_data(name= 'val',x= x_val,y=y_val)
# data.save_data(name= 'test',x= x_test,y=y_test)

# Test model

In [1]:
from models import base_model as test_model

In [8]:
# Model parametrs
class CN_default(DataClass):
    n_filters = 14
    dilation_steps = 4  # dilation = 2**dilation_step


class AN_default(DataClass):
    attention_heads = 3
    blocks = 2
    share_weights = False


class FF_default(DataClass):
    dropout_rate = 0.1


class Optimaser_default(DataClass):
    lr = 0.0001
    adam_beta1 = 0.9
    adam_beta2 = 0.999


class Trainig_default(DataClass):
    shuffle = True
    batch_size = 512
    epochs = 150


class Full_default(DataClass):
    cn = CN_default()
    an = AN_default()
    ff = FF_default()
    optimizer = keras.optimizers.Adam(
        learning_rate=0.0001,
        beta_1=0.9,
        beta_2=0.999,
        name="Adam",
    )
    training = Trainig_default()


default_pars = Full_default()

In [9]:
# Datasets
datasets = data.build_datasets(
    datas=row_data,
    batch_size=default_pars.training.batch_size,
    seq_len=seq_len,
)
ds_train, ds_val, ds_test = datasets['train'], datasets['val'], datasets[
    'test']
data.inspect_datasets(datasets)

    Datasets:
train     : [398, 100, 40]
val       : [100, 100, 40]
test      : [273, 100, 40]


In [10]:
# Build
model = test_model.build_model()

In [11]:
# Train
model.fit(
    ds_train,
    epochs=default_pars.training.epochs,
    validation_data=ds_val,
    # callbacks=[
    #     CSVLogger(r'logs/model1/log.csv', append=True, separator=';')
    # ]
)

Epoch 1/150
  5/398 [..............................] - ETA: 6:33 - loss: 2.4189 - accurancy: 0.5191 - sparce_accurancy: 0.3578

KeyboardInterrupt: 

# Search

In [3]:
from keras_tuner.tuners import GridSearch
import keras_tuner

import models

In [4]:
# Datasets
datasets = data.build_datasets(
    datas=row_data,
    batch_size=512,
    seq_len=seq_len,
)
ds_train, ds_val, ds_test = datasets['train'], datasets['val'], datasets[
    'test']
data.inspect_datasets(datasets)

    Datasets:
train     : [398, 100, 40]
val       : [100, 100, 40]
test      : [273, 100, 40]


In [5]:
def configure(hp: keras_tuner.HyperParameters):

    class CN_search(DataClass):
        dilation_steps = hp.Int(
            'cn_layers',
            default=4,
            min_value=3,
            max_value=5,
            step=1,
        )

    class AN_search(DataClass):
        attention_heads = 3
        blocks = 2
        share_weights = hp.Boolean(
            'share_weights',
            default=True,
        )

    class FF_search(DataClass):
        dropout_rate = hp.Float(
            'dropout_rate',
            default=0.1,
            min_value=0,
            max_value=0.5,
            step=0.1,
        )

    lr = hp.Float(
        'lr',
        default=1e-4,
        min_value=1e-6,
        max_value=1e-3,
    )
    beta1 = hp.Float(
        'beta1',
        default=0.9,
        min_value=0.5,
        max_value=1.1,
        step=0.1,
    )
    beta2 = hp.Float(
        'beta1',
        default=0.999,
        min_value=0.5,
        max_value=1.1,
        sampling='log',
    )

    class Full_search(DataClass):
        cn = CN_search()
        an = AN_search()
        ff = FF_search()
        optimizer = keras.optimizers.Adam(
            name="Adam",
            learning_rate=lr,
            beta_1=beta1,
            beta_2=beta2,
        )

    return Full_search()


a = configure(keras_tuner.HyperParameters())
a.Info_expanded

{'cn__dilation_steps': 4,
 'an__attention_heads': 3,
 'an__blocks': 2,
 'an__share_weights': True,
 'ff__dropout_rate': 0.1,
 'optimizer': <keras.src.optimizers.adam.Adam at 0x2783d93e510>}

In [15]:
def search_base_model(hp):
    a = configure(hp)
    model = models.base_model.build_model(**a.Info_expanded)
    return model


In [16]:
tuner = keras_tuner.RandomSearch(
    hypermodel=search_base_model,
    objective="val_accuracy",
    max_trials=3,
    executions_per_trial=2,
    directory="search",
    project_name="helloworld",
)

INFO:tensorflow:Reloading Tuner from my_dir\helloworld\tuner0.json


In [17]:
tuner.search(
    ds_train,
    epochs=2,
)



Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
4                 |4                 |cn_layers
16                |16                |cn_filters
False             |False             |share_weights
0.4               |0.4               |dropout_rate

Epoch 1/2
  4/398 [..............................] - ETA: 6:21 - loss: 2.7428 - accurancy: 0.1353 - sparce_accurancy: 0.2998

KeyboardInterrupt: 